In [ ]:
'''
https://www.kaggle.com/code/aiinradiology/rsna-data-handling
'''

In [1]:
#import relevant libraries:

import pandas as pd
import numpy as np
import os as os
import matplotlib.pyplot as plt #For ploting and working with graphs
#import seaborn as sns
import cv2 #Computer vision version 2 library for loading and reading images
import pydicom #for working wiht dicom images (MRI images are dicom images with .dcm extension)
import os # Already imported above. This Operating system library is to work with computer kernel.usefyl to interact with computer kernel and run commands like terminal. 
import glob #glob library helps finding global patterns in file names. Not sure how it will be useful !
from tqdm import tqdm #Taqadum is arabic wor meanng progress and hence tqdm is ibrary which help in showing progress bars.
import warnings #lbraary helps in rasing warning messages where needed. 
import ipywidgets as widgets
import pickle
from IPython.display import display
import logging
from multiprocessing import Pool
import bz2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

In [2]:
import tensorflow as tf
if (len(tf.config.list_physical_devices('GPU'))==1):
    print("TensorFlow has access to the GPU.")

TensorFlow has access to the GPU.


In [3]:
gpus=tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            print("GPU is available and configured sucesssfully")
    except RuntimeError as e:
        print(e)
else:
    print('No GPUS found!')

GPU is available and configured sucesssfully


In [4]:
#테스트 코드를 실행하여 Tensorflow가 GPU를 사용하고 있는지 확인합니다:

In [5]:
with tf.device('/GPU:0'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    c = tf.matmul(a, b, transpose_b=True)  # Correct the matrix dimensions for matmul
    print(c.numpy())

[[14. 32.]
 [32. 77.]]


In [6]:
path='spine/'
train_data_path=path + 'train.csv'
sample_csv_path=path + 'sample_submission.csv'
train_series_descriptions_path=path + 'train_series_descriptions.csv'
train_label_coordinates_data_path=path + 'train_label_coordinates.csv'
test_series_descriptions_path=path + 'test_series_descriptions.csv'

In [7]:
train_images_path=path + 'train_images'
test_images_path=path + 'test_images'

In [8]:
sample_csv=pd.read_csv(sample_csv_path)
train_data=pd.read_csv(train_data_path)
train_series_description=pd.read_csv(train_series_descriptions_path)
test_series_description=pd.read_csv(test_series_descriptions_path)
train_label_coordinates_data=pd.read_csv(train_label_coordinates_data_path)

In [9]:
train_data

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1,4646740,Normal/Mild,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate,Normal/Mild
2,7143189,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
3,8785691,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4,10728036,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,4282019580,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate
1971,4283570761,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1972,4284048608,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild
1973,4287160193,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Severe,Moderate,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild


In [10]:
#Trun f warnings!
warnings.filterwarnings("ignore")

In [11]:
train_label_coordinates_data

,study_id,series_id,instance_number,condition,level,x,y
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602
...,...,...,...,...,...,...,...
48687,4290709089,4237840455,11,Left Neural Foraminal Narrowing,L1/L2,219.465940,97.831063
48688,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L2/L3,205.340599,140.207084
48689,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L3/L4,202.724796,181.013624
48690,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L4/L5,202.933333,219.733333


In [12]:
train_label_coordinates_data["category"] = None
train_label_coordinates_data

,study_id,series_id,instance_number,condition,level,x,y,category
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,None
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,None
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,None
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,None
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,None
...,...,...,...,...,...,...,...,...
48687,4290709089,4237840455,11,Left Neural Foraminal Narrowing,L1/L2,219.465940,97.831063,None
48688,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L2/L3,205.340599,140.207084,None
48689,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L3/L4,202.724796,181.013624,None
48690,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L4/L5,202.933333,219.733333,None


In [13]:
train_label_coordinates_data.iterrows()

<generator object DataFrame.iterrows at 0x000001E7BD5DCCF0>

In [14]:
for idx, row in train_label_coordinates_data.iterrows():
    r = row['study_id']
    col = (row['condition'].lower().replace(' ', '_')) + '_' + (row['level'].lower().replace('/', '_'))
    
    # Ensure the column exists in train_data to avoid KeyError
    if col in train_data.columns:
        # Check if there is a matching study_id and the column exists
        value = train_data.loc[train_data['study_id'] == r, col].values
        if len(value) > 0:
            train_label_coordinates_data.at[idx, "category"] = value[0]
        else:
            train_label_coordinates_data.at[idx, "category"] = None
    else:
        train_label_coordinates_data.at[idx, "category"] = None

In [15]:
train_label_coordinates_data

,study_id,series_id,instance_number,condition,level,x,y,category
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Normal/Mild
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Normal/Mild
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Normal/Mild
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Normal/Mild
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Normal/Mild
...,...,...,...,...,...,...,...,...
48687,4290709089,4237840455,11,Left Neural Foraminal Narrowing,L1/L2,219.465940,97.831063,Normal/Mild
48688,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L2/L3,205.340599,140.207084,Normal/Mild
48689,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L3/L4,202.724796,181.013624,Normal/Mild
48690,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L4/L5,202.933333,219.733333,Normal/Mild


In [16]:
# 이제 이미지 파일 경로 목록을 만들어야 합니다:
# 이를 위해 먼저 처음 세 개의 열만 포함하는 새로운 df를 만들겠습니다. 
# 우리가 해야 할 일은 나머지 열을 제거하는 것입니다.

In [17]:
first_three_cols=train_label_coordinates_data.iloc[:,:3]
first_three_cols

,study_id,series_id,instance_number
0,4003253,702807833,8
1,4003253,702807833,8
2,4003253,702807833,8
3,4003253,702807833,8
4,4003253,702807833,8
...,...,...,...
48687,4290709089,4237840455,11
48688,4290709089,4237840455,12
48689,4290709089,4237840455,12
48690,4290709089,4237840455,12


In [18]:
# 이제 우리는 이 df를 각 행의 결합된 값을 포함하는 새로운 df로 변환하기만 하면 됩니다.

In [19]:
join_row_as_str=lambda row: train_images_path+"/"+"/".join(row.astype(str))+".dcm"
train_label_images_path=first_three_cols.apply(join_row_as_str,axis=1)
train_label_images_path

0             spine/train_images/4003253/702807833/8.dcm
1             spine/train_images/4003253/702807833/8.dcm
2             spine/train_images/4003253/702807833/8.dcm
3             spine/train_images/4003253/702807833/8.dcm
4             spine/train_images/4003253/702807833/8.dcm
                              ...                       
48687    spine/train_images/4290709089/4237840455/11.dcm
48688    spine/train_images/4290709089/4237840455/12.dcm
48689    spine/train_images/4290709089/4237840455/12.dcm
48690    spine/train_images/4290709089/4237840455/12.dcm
48691    spine/train_images/4290709089/4237840455/12.dcm
Length: 48692, dtype: object

In [20]:
print(f'The number of rows in train_label_images_path and train_label_coordinates_data are equal? {train_label_images_path.shape[0]==train_label_coordinates_data.shape[0]}')

The number of rows in train_label_images_path and train_label_coordinates_data are equal? True


In [21]:
train_label_coordinates_data["img_file_path"] = None
train_label_coordinates_data

,study_id,series_id,instance_number,condition,level,x,y,category,img_file_path
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Normal/Mild,None
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Normal/Mild,None
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Normal/Mild,None
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Normal/Mild,None
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Normal/Mild,None
...,...,...,...,...,...,...,...,...,...
48687,4290709089,4237840455,11,Left Neural Foraminal Narrowing,L1/L2,219.465940,97.831063,Normal/Mild,None
48688,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L2/L3,205.340599,140.207084,Normal/Mild,None
48689,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L3/L4,202.724796,181.013624,Normal/Mild,None
48690,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L4/L5,202.933333,219.733333,Normal/Mild,None


In [22]:
for idx, row in train_label_coordinates_data.iterrows():
    file_path = train_label_images_path.loc[idx]
    train_label_coordinates_data.at[idx, "img_file_path"] = file_path
train_label_coordinates_data   

,study_id,series_id,instance_number,condition,level,x,y,category,img_file_path
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Normal/Mild,spine/train_images/4003253/702807833/8.dcm
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Normal/Mild,spine/train_images/4003253/702807833/8.dcm
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Normal/Mild,spine/train_images/4003253/702807833/8.dcm
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Normal/Mild,spine/train_images/4003253/702807833/8.dcm
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Normal/Mild,spine/train_images/4003253/702807833/8.dcm
...,...,...,...,...,...,...,...,...,...
48687,4290709089,4237840455,11,Left Neural Foraminal Narrowing,L1/L2,219.465940,97.831063,Normal/Mild,spine/train_images/4290709089/4237840455/11.dcm
48688,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L2/L3,205.340599,140.207084,Normal/Mild,spine/train_images/4290709089/4237840455/12.dcm
48689,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L3/L4,202.724796,181.013624,Normal/Mild,spine/train_images/4290709089/4237840455/12.dcm
48690,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L4/L5,202.933333,219.733333,Normal/Mild,spine/train_images/4290709089/4237840455/12.dcm


In [23]:
train_data = train_label_coordinates_data

In [24]:
# Check for missing values
display(train_data.isnull().sum())

study_id            0
series_id           0
instance_number     0
condition           0
level               0
x                   0
y                   0
category           35
img_file_path       0
dtype: int64

In [25]:
#누락된 범주를 시각화하여 누락된 내용을 파악합니다:

In [26]:
missing_category_entries = train_data[train_data['category'].isnull()]
display(missing_category_entries.head(2))
print(f"Number of rows with missing 'category' entries: {len(missing_category_entries)}\n----\n")

,study_id,series_id,instance_number,condition,level,x,y,category,img_file_path
5364,462494704,682162323,9,Right Neural Foraminal Narrowing,L2/L3,260.974125,229.893455,NaN,spine/train_images/462494704/682162323/9.dcm
5365,462494704,682162323,9,Right Neural Foraminal Narrowing,L3/L4,244.608828,282.106545,NaN,spine/train_images/462494704/682162323/9.dcm


Number of rows with missing 'category' entries: 35
----



In [27]:
# '카테고리'가 null인 고유 스터디 ID 가져오기

In [28]:
unique_study_ids_missing_category_entries = missing_category_entries['study_id'].unique()
unique_study_ids_missing_category_entries

array([ 462494704,  482624307,  779042451, 1524089207, 2507107985,
       2607462358, 3781188430], dtype=int64)

In [29]:
num_unique_study_ids_missing_category_entries = len(unique_study_ids_missing_category_entries)
num_unique_study_ids_missing_category_entries

7

In [30]:
print(f"Number of unique study IDs with missing 'category' entries: {num_unique_study_ids_missing_category_entries}")
display(unique_study_ids_missing_category_entries)

Number of unique study IDs with missing 'category' entries: 7


array([ 462494704,  482624307,  779042451, 1524089207, 2507107985,
       2607462358, 3781188430], dtype=int64)

In [31]:
train= train_data.dropna(inplace=False)

In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48657 entries, 0 to 48691
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   study_id         48657 non-null  int64  
 1   series_id        48657 non-null  int64  
 2   instance_number  48657 non-null  int64  
 3   condition        48657 non-null  object 
 4   level            48657 non-null  object 
 5   x                48657 non-null  float64
 6   y                48657 non-null  float64
 7   category         48657 non-null  object 
 8   img_file_path    48657 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 3.7+ MB


In [33]:
def preprocess_images(df, image_column, target_size, num_of_rows=1000):
    """
    Function to resize and normalize the images in a dataset, and serialize them using pickle.

    Args:
    df (DataFrame): The dataset containing the path to images.
    image_column (str): The name of the column in df which contains the image file paths.
    target_size (tuple): The desired target size of the image (usually (224, 224)).
    num_of_rows (int): An integer that defines how many rows are processed at a time. 
                       By default, it is set to 1000. To process the entire dataset, set this to -1.

    Returns:
    list: A list of serialized processed images that can be appended to any df as a new column.

    Steps Involved:
    1. Loop through the specified number of rows in the dataframe.
    2. Read each DICOM file using pydicom.
    3. Extract the image data from the DICOM file.
    4. Convert the image to a format compatible with OpenCV.
    5. Resize the image using OpenCV.
    6. Normalize pixel values.
    7. Serialize the processed image using pickle.
    8. Append the serialized image to a list.
    9. Return the list of serialized processed images.

    Example usage:
    processed_images = preprocess_images(train, 'image_file_path', (224, 224))

    To save the processed images to a DataFrame:
    temp_train = train.copy()
    temp_train.loc[:999, 'processed_images'] = processed_images

    To save the DataFrame to a pickle file:
    temp_train.to_pickle('processed_train_data.pkl')


    데이터 세트의 이미지 크기를 조정하고 정규화하며 피클을 사용하여 이미지를 직렬화하는 기능을 수행합니다.

Args:
df(DataFrame): 이미지로의 경로를 포함하는 데이터 세트입니다.
image_column(str): 이미지 파일 경로를 포함하는 df의 열 이름입니다.
target_size (tuple): 이미지의 원하는 타겟 크기 (보통 (224, 224)).
num_of_rows(int): 한 번에 처리되는 행의 수를 정의하는 정수입니다.
기본적으로 1000으로 설정됩니다. 전체 데이터 세트를 처리하려면 -1로 설정합니다.

반환:
list: df에 새 열로 추가할 수 있는 직렬 처리된 이미지 목록입니다.

관련 단계:
1. 데이터 프레임에서 지정된 수의 행을 반복합니다.
2. Pydicom을 사용하여 각 DICOM 파일을 읽습니다.
3. DICOM 파일에서 영상 데이터를 추출합니다.
4. OpenCV와 호환되는 형식으로 이미지를 변환합니다.
5. OpenCV를 사용하여 영상 크기를 조정합니다.
6. 픽셀 값을 정규화합니다.
7. 피클을 사용하여 처리된 이미지를 직렬화합니다.
8. 직렬화된 이미지를 목록에 추가합니다.
9. 직렬화된 처리된 이미지 목록을 반환합니다.

사용 예:
processed_images =preprocess_images(train, 'image_file_path', (224, 224))

처리된 이미지를 DataFrame에 저장하려면:
temp_train = train.copy ()
temp_train.loc[:999, 'processed_images'] = 처리된_images

DataFrame을 피클 파일에 저장하려면:
temp_train.to_pickle ('processed_train_data.pkl')
    """

    processed_images = []
    
    if num_of_rows == -1:
        num_of_rows = len(df)
    
    for image_path in tqdm(df[:num_of_rows][image_column]):
        try:
            # Read the DICOM file using pydicom
            dicom = pydicom.dcmread(image_path)
            
            # # DICOM 파일에서 영상 데이터 추출
            image = dicom.pixel_array
            
            # OpenCV와 호환되는 형식으로 이미지 변환
            image = cv2.convertScaleAbs(image)
            
            # OpenCV를 사용하여 영상 크기 조정
            image = cv2.resize(image, target_size)
            
            # 픽셀 값 정규화
            image = image / 255.0
            
            # 처리된 이미지를 직렬화
            serialized_image = pickle.dumps(image)
            processed_images.append(serialized_image)
        except Exception as e:
            print(f"Error processing file {image_path}: {e}")
    
    print(f"Successfully processed images in {len(df[:num_of_rows])} rows")
    return processed_images

In [34]:
processed_images=preprocess_images(train,"img_file_path",(224,224),-1)

100%|████████████████████████████████████████████████████████████████████████████| 48657/48657 [08:07<00:00, 99.82it/s]

Successfully processed images in 48657 rows


In [55]:
help(preprocess_images)

Help on function preprocess_images in module __main__:

preprocess_images(df, image_column, target_size, num_of_rows=1000)
        Function to resize and normalize the images in a dataset, and serialize them using pickle.
    
        Args:
        df (DataFrame): The dataset containing the path to images.
        image_column (str): The name of the column in df which contains the image file paths.
        target_size (tuple): The desired target size of the image (usually (224, 224)).
        num_of_rows (int): An integer that defines how many rows are processed at a time. 
                           By default, it is set to 1000. To process the entire dataset, set this to -1.
    
        Returns:
        list: A list of serialized processed images that can be appended to any df as a new column.
    
        Steps Involved:
        1. Loop through the specified number of rows in the dataframe.
        2. Read each DICOM file using pydicom.
        3. Extract the image data from the DI

In [37]:
print(len(processed_images))

48657


In [38]:
#Lets add processed images as new column in the our dataset.

# Create a temporary DataFrame to store the processed images
processed_train = train.copy()
#check if the len of the processed_images and the number of rows in processed_train is equal or not.
print(f'Is number of procssed images in processed_images is = number of rows in processed_train\ndataframe?\n-:{len(processed_images)==processed_train.shape[0]}.')

Is number of procssed images in processed_images is = number of rows in processed_train
dataframe?
-:True.


In [39]:
processed_train['processed_images'] = processed_images

In [40]:
#Check the updated dataset:
display(processed_train.head(3))
print(processed_train.shape)

,study_id,series_id,instance_number,condition,level,x,y,category,img_file_path,processed_images
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Normal/Mild,spine/train_images/4003253/702807833/8.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Normal/Mild,spine/train_images/4003253/702807833/8.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Normal/Mild,spine/train_images/4003253/702807833/8.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...


(48657, 10)


In [41]:
from sklearn.preprocessing import LabelEncoder

# Fit label encoder to your training labels
le = LabelEncoder()
le.fit(processed_train['category'])

LabelEncoder()

In [42]:
le.classes_

array(['Moderate', 'Normal/Mild', 'Severe'], dtype=object)

In [43]:
processed_train['category'] = le.transform(processed_train['category'])

In [44]:
#Check the updated dataset:
display(processed_train.head(3))
print(processed_train.shape)

,study_id,series_id,instance_number,condition,level,x,y,category,img_file_path,processed_images
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,1,spine/train_images/4003253/702807833/8.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,1,spine/train_images/4003253/702807833/8.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,1,spine/train_images/4003253/702807833/8.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...


(48657, 10)


In [45]:
#check the laeblecnoding : Remeebr we named Labelenvoder as le before fitting it to outr classes-:
le_map=dict(zip(sorted(le.classes_),le.transform(le.classes_)))
print(le_map)
# See how mny. ategories (labels) are there and what count of each category ?
category_counts=processed_train['category'].value_counts()
print(category_counts)

{'Moderate': 0, 'Normal/Mild': 1, 'Severe': 2}
category
1    37626
0     7950
2     3081
Name: count, dtype: int64


In [48]:
from sklearn.model_selection import train_test_split

# Split the data
train_df, val_df = train_test_split(processed_train, test_size=0.2, random_state=42, stratify=processed_train['category'])

In [49]:
# DCheck if the train and val df are in good shape:

print("Showing train_df\n-----------n")
display(train_df.head())
print(train_df.shape)

print("Showing val_df\n-----------n")
display(val_df.head())
print(val_df.shape)

Showing train_df
-----------n


,study_id,series_id,instance_number,condition,level,x,y,category,img_file_path,processed_images
46219,4075603869,1361414844,8,Spinal Canal Stenosis,L3/L4,180.215054,145.519713,1,spine/train_images/4075603869/1361414844/8.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
3641,314041963,277371578,12,Spinal Canal Stenosis,L1/L2,376.057300,114.386185,1,spine/train_images/314041963/277371578/12.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
25382,2288221191,1504692102,13,Right Subarticular Stenosis,L2/L3,292.706422,343.905636,1,spine/train_images/2288221191/1504692102/13.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
45671,4029974537,2684154860,37,Right Subarticular Stenosis,L5/S1,195.500000,250.128440,1,spine/train_images/4029974537/2684154860/37.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
47246,4163587601,3190560134,41,Right Subarticular Stenosis,L5/S1,142.390671,161.399417,0,spine/train_images/4163587601/3190560134/41.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...


(38925, 10)
Showing val_df
-----------n


,study_id,series_id,instance_number,condition,level,x,y,category,img_file_path,processed_images
11118,984887785,3830185810,6,Left Neural Foraminal Narrowing,L4/L5,161.786988,197.361854,2,spine/train_images/984887785/3830185810/6.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
44346,3895795003,4014059471,7,Spinal Canal Stenosis,L4/L5,144.424779,165.522124,1,spine/train_images/3895795003/4014059471/7.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
32761,2888359875,2906456873,13,Left Subarticular Stenosis,L4/L5,274.669492,257.084746,2,spine/train_images/2888359875/2906456873/13.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
43677,3859825215,3632938463,13,Left Neural Foraminal Narrowing,L5/S1,239.483871,303.362429,0,spine/train_images/3859825215/3632938463/13.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
22510,2026420722,2823687618,4,Left Neural Foraminal Narrowing,L4/L5,175.771785,324.564481,1,spine/train_images/2026420722/2823687618/4.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...


(9732, 10)


In [50]:
val_df.to_pickle('val_data.pkl')

In [51]:
# Check if proper pickling done or not for val_df
val_check = pd.read_pickle('val_data.pkl')

if val_df.equals(val_check):
    print("The val_df has been properly pickled.")
else:
    print("There was an error in pickling the val_df.")

The val_df has been properly pickled.


In [53]:
train_df.to_pickle('train_data.pkl')

In [55]:
train_check = pd.read_pickle('train_data.pkl')

if train_df.equals(train_check):
    print("The val_df has been properly pickled.")
else:
    print("There was an error in pickling the val_df.")

The val_df has been properly pickled.


In [56]:
val_check

,study_id,series_id,instance_number,condition,level,x,y,category,img_file_path,processed_images
11118,984887785,3830185810,6,Left Neural Foraminal Narrowing,L4/L5,161.786988,197.361854,2,spine/train_images/984887785/3830185810/6.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
44346,3895795003,4014059471,7,Spinal Canal Stenosis,L4/L5,144.424779,165.522124,1,spine/train_images/3895795003/4014059471/7.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
32761,2888359875,2906456873,13,Left Subarticular Stenosis,L4/L5,274.669492,257.084746,2,spine/train_images/2888359875/2906456873/13.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
43677,3859825215,3632938463,13,Left Neural Foraminal Narrowing,L5/S1,239.483871,303.362429,0,spine/train_images/3859825215/3632938463/13.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
22510,2026420722,2823687618,4,Left Neural Foraminal Narrowing,L4/L5,175.771785,324.564481,1,spine/train_images/2026420722/2823687618/4.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
...,...,...,...,...,...,...,...,...,...,...
29242,2604747522,3379345892,5,Right Neural Foraminal Narrowing,L4/L5,277.886334,299.838692,0,spine/train_images/2604747522/3379345892/5.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
42591,3748910433,3587805634,28,Right Subarticular Stenosis,L3/L4,148.331601,168.648649,1,spine/train_images/3748910433/3587805634/28.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
31615,2797118205,2740894765,12,Right Neural Foraminal Narrowing,L2/L3,150.511775,151.449275,0,spine/train_images/2797118205/2740894765/12.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
9566,859269912,2394702425,4,Right Neural Foraminal Narrowing,L5/S1,288.861759,361.163375,1,spine/train_images/859269912/2394702425/4.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...


In [57]:
train_check

,study_id,series_id,instance_number,condition,level,x,y,category,img_file_path,processed_images
46219,4075603869,1361414844,8,Spinal Canal Stenosis,L3/L4,180.215054,145.519713,1,spine/train_images/4075603869/1361414844/8.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
3641,314041963,277371578,12,Spinal Canal Stenosis,L1/L2,376.057300,114.386185,1,spine/train_images/314041963/277371578/12.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
25382,2288221191,1504692102,13,Right Subarticular Stenosis,L2/L3,292.706422,343.905636,1,spine/train_images/2288221191/1504692102/13.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
45671,4029974537,2684154860,37,Right Subarticular Stenosis,L5/S1,195.500000,250.128440,1,spine/train_images/4029974537/2684154860/37.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
47246,4163587601,3190560134,41,Right Subarticular Stenosis,L5/S1,142.390671,161.399417,0,spine/train_images/4163587601/3190560134/41.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
...,...,...,...,...,...,...,...,...,...,...
15268,1367470327,3111450149,8,Spinal Canal Stenosis,L2/L3,181.012195,127.219512,1,spine/train_images/1367470327/3111450149/8.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
44668,3931051195,3232004939,8,Spinal Canal Stenosis,L4/L5,304.426330,453.135238,1,spine/train_images/3931051195/3232004939/8.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
48389,4266523380,3703649431,15,Left Neural Foraminal Narrowing,L2/L3,165.017218,156.914601,1,spine/train_images/4266523380/3703649431/15.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
4076,344269999,2933376913,7,Spinal Canal Stenosis,L1/L2,321.807910,163.917137,1,spine/train_images/344269999/2933376913/7.dcm,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
